<a href="https://colab.research.google.com/github/lifepopkay/Tech-Monies/blob/Data-cleaning/Final_Data_cleanup_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final script used for cleaning the scrapped dataset from indeed

### Import packages

In [1]:
#import packages needed
import pandas as pd
import numpy as np
import re
import requests as rq

### Get the raw data from onedrive

In [2]:
# data url
url = "https://drive.google.com/u/0/uc?id=1Mx6ilfA0Ffm1s9ff1lgAiUnK8DxAPPBt&export=download"

# get response from the URL
response = rq.get(url)

# write content to excel file
with open("dataset.xlsx", "wb") as file:
    file.write(response.content)

# read data
data = pd.read_excel("dataset.xlsx")

data.head(5)

,title,id,company,location,link,salaryDesc,postDate,JobDesc,title scraped for,Country
0,Senior Software Engineer,job_3e1d31dda6c5cb4b,Offerzen,Lagos,/company/OfferZen/jobs/Senior-Software-Enginee...,Full-time,Just posted,The Opportunity\nOur newly formed Marketplace ...,Data Scientist,Nigeria
1,Data Scientist,job_c3e2ed6ba483528b,GVA Partners,Lagos,/company/GVA-Partners/jobs/Data-Scientist-c3e2...,Permanent +1,Today,Our client requires the skills of a Data Scien...,Data Scientist,Nigeria
2,Full stack developer,job_264deaa0926c095e,Horizonpay Nigeria Limited,Lagos,/company/HorizonPay-Nigeria-Limited/jobs/Full-...,"‚Ç¶700,000 a monthFull-time",Today,\nWork with development teams and product mana...,Data Scientist,Nigeria
3,Superintendent Pharmaceutical Officer,job_4ba7daa713f2751c,NaN,Lagos,/rc/clk?jk=4ba7daa713f2751c&fccid=dd616958bd9d...,Full-time,4 days ago,\n\n\n\nSuperintendent Pharmaceutical Officer\...,Data Scientist,Nigeria
4,Project HSE Advisor I,job_45b19f53adcf0053,Worley,Abeokuta,/rc/clk?jk=45b19f53adcf0053&fccid=d9805af20a6c...,Full-time,5 days ago,\n\n Company : Worley \n \n\nPrimary Location...,Data Scientist,Nigeria


### Salary and pay frequency Extraction
Extraction functions for pay frequency, and salary into upper & lower range columns

In [3]:
# functions 
def upper_salary_range(x):
    
    """
    This function extracts the upper salary range from the scrapped salary description and returns the upper salary.
    
    arg :
        x : the salary description scrapped (str)
    
    return:
        upper_salary_value : cleaned upper salary value (int)
    """
    
    scrapped_salary = str(x).split()
    upper_salary_value = ''
    lower_salary_value = ''
    trackposition = 0
    digit = ['0','1','2','3','4','5','6','7','8','9']
    
    for item in scrapped_salary:
        text_check = False
        #checking if the item in the list is possibliy salary( confirming if the 3 item is a number)
       
        
        try:
            text_check1= item[1]
            if text_check1 in digit and item[0] != '+':
                text_check = True
            else:
                try:
                    text_check2 = item[2]
                    if text_check2 in digit and item[0] != '+':
                        text_check = True
                    else:
                        try:
                            text_check3= item[3]
                            if text_check3 in digit and item[0] != '+':
                                text_check = True
                        except:
                            text_check = False
                except:
                    text_check = False
        except:
            text_check = False
            
            
        if trackposition == 0 and text_check == True:
            #lower_salary_value = t
             #remove leading letters from salary
            if item[0] in digit:
                lower_salary_value = item
            elif item[1] in digit:
                lower_salary_value = item[1:]
            elif item[2] in digit:
                lower_salary_value = item[2:]
            elif item[3] in digit:
                lower_salary_value = item[3:]
            
            # change tracker to shower lower_range is picked
            trackposition = 1    
            
            # find lower_salary_value range with upper_salary_value mergered togther
            for i in lower_salary_value:
                if i != ',' and i.lower() != 'k'and i != '.' and i not in digit:
                    ind = lower_salary_value.index(i)
                    upper_salary_value = lower_salary_value[ind:]
                    lower_salary_value = ''
                    
                    #clean currency from upper_salary_value
                    if upper_salary_value[0] in digit:
                        upper_salary_value = upper_salary_value
                    elif upper_salary_value[1] in digit:
                        upper_salary_value = upper_salary_value[1:]
                    elif upper_salary_value[2] in digit:
                        upper_salary_value = upper_salary_value[2:]
                    elif upper_salary_value[3] in digit:
                        upper_salary_value = upper_salary_value[3:]
                    elif upper_salary_value[4] in digit:
                        upper_salary_value = upper_salary_value[4:]
                    elif upper_salary_value[5] in digit:
                        upper_salary_value = upper_salary_value[5:]
                    
                    trackposition = 0  # toggle back tracker as upper has been picked
                    #exit the loop
                    break
                
            
        elif trackposition == 1 and text_check == True:
            if item[0] in digit and lower_salary_value != '':
                upper_salary_value = item
            elif item[1] in digit and lower_salary_value != '':
                upper_salary_value = item[1:]
            elif item[2] in digit and lower_salary_value != '':
                upper_salary_value = item[2:]
            elif item[3] in digit and lower_salary_value != '':
                upper_salary_value = item[3:]
                
                
     # replace ',' with '' 
    upper_salary_value = upper_salary_value.replace(',', '')
    
    #replace 'k' with '000'
    upper_salary_value = upper_salary_value.upper()
    if upper_salary_value[-1:] == 'K':
        upper_salary_value = upper_salary_value.replace('K', '')
        upper_salary_value = float(upper_salary_value)*1000
    
    #convert to int
    try:
        upper_salary_value = float(upper_salary_value)
    except:
        upper_salary_value = np.nan
    
    
    return upper_salary_value


def lower_salary_range(x):
        
    """
    This function extracts the lower salary range from the scrapped salary description and returns the lower salary.
    
    arg :
        x : the salary description scrapped (str)
    
    return:
        lower_salary_value : cleaned upper salary value (int)
    """
    scrapped_salary = str(x).split()   #split the salary description
    upper_salary_value = ''     #variable for holder upper salary range
    lower_salary_value = ''     #variable for holder lower salary range
    trackposition = 0        #variable for tracking when lower salary has been extracted 

    digit = ['0','1','2','3','4','5','6','7','8','9']  # number listing
    
    #looping through the description for the extraction 
    for item in scrapped_salary:
        text_check = False
        #checking if the item in the list is possibliy salary( confirming if the 3 item is a number)
       
        
        try:
            text_check1= item[1]
            if text_check1 in digit and item[0] != '+':
                text_check = True
            else:
                try:
                    text_check2 = item[2]
                    if text_check2 in digit and item[0] != '+':
                        text_check = True
                    else:
                        try:
                            text_check3= item[3]
                            if text_check3 in digit and item[0] != '+':
                                text_check = True
                        except:
                            text_check = False
                except:
                    text_check = False
        except:
            text_check = False
            
        
            
        
        #extraction based on tracker
        if trackposition == 0 and text_check == True:
            #remove leading letters from salary
            if item[0] in digit:
                lower_salary_value = item
            elif item[1] in digit:
                lower_salary_value = item[1:]
            elif item[2] in digit:
                lower_salary_value = item[2:]
            elif item[3] in digit:
                lower_salary_value = item[3:]
                
             # find lower range with upper mergered togther
            for i in lower_salary_value:
                if i != ',' and i.lower() != 'k' and i != '.'and i not in digit:
                    ind = lower_salary_value.index(i)
                    lower_salary_value = lower_salary_value[:ind]
                    upper_salary_value = lower_salary_value[ind:]
                    break
            
            #Update tracker 
            trackposition = 1
            
        elif trackposition == 1 and text_check == True:
            upper_salary_value = item
    
    # replace ',' with '' 
    lower_salary_value = lower_salary_value.replace(',', '')
    
    #and 'k' with '000'
    lower_salary_value = lower_salary_value.upper()
    if lower_salary_value[-1:] == 'K':
        lower_salary_value = lower_salary_value.replace('K', '')
        lower_salary_value = float(lower_salary_value)*1000
    
    #convert to int
    try:
        lower_salary_value = float(lower_salary_value)
    except:
        lower_salary_value = np.nan
        
    return lower_salary_value


def pay_freq(x):  
    """
    This function extracts the payment frequency from the scrapped salary description.
    
    arg :
        x : the salary description scrapped (str)
    
    return:
        results : payment frequency (str)
    """
    text = str(x).split()
    month_re = r'month\w*'
    year_re = r'year\w*'
    hour_re = r'hour\w*'

    results = None

    for t in text:
        month_finder = re.findall(month_re, t.lower())
        year_finder = re.findall(year_re, t.lower())
        hour_finder = re.findall(hour_re, t.lower())
        if len(year_finder) > 0:
            results = 'Yearly'
            break
        elif len(month_finder) > 0:
            results = 'Monthly'
            break
        elif len(hour_finder) > 0:
            results = 'Hourly'
    return results


#create and extract the upper and lower salary range for each row
data['upper_salary'] = data['salaryDesc'].apply(lambda x: upper_salary_range(x))
data['lower_salary'] = data['salaryDesc'].apply(lambda x: lower_salary_range(x))
data['payment_frequency'] = data['salaryDesc'].apply(lambda x: pay_freq(x))


data[['upper_salary', 'lower_salary','payment_frequency']].sample(20)

,upper_salary,lower_salary,payment_frequency
1765,1521644.0,1060843.0,Yearly
6014,118300.0,66100.0,Yearly
2442,NaN,NaN,None
3189,NaN,NaN,None
6785,NaN,NaN,None
3179,NaN,NaN,None
4286,NaN,NaN,None
1529,NaN,NaN,None
7571,NaN,NaN,None
4731,NaN,25000.0,Monthly


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8700 entries, 0 to 8699
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              8700 non-null   object 
 1   id                 8675 non-null   object 
 2   company            8574 non-null   object 
 3   location           8675 non-null   object 
 4   link               8675 non-null   object 
 5   salaryDesc         6589 non-null   object 
 6   postDate           8675 non-null   object 
 7   JobDesc            8675 non-null   object 
 8   title scraped for  8700 non-null   object 
 9   Country            8700 non-null   object 
 10  upper_salary       2397 non-null   float64
 11  lower_salary       2833 non-null   float64
 12  payment_frequency  2808 non-null   object 
dtypes: float64(2), object(11)
memory usage: 883.7+ KB


### Years of Experience Extraction
Extraction of years of experience 

In [15]:
def years_exp(x):
  global result
  expression = "\d*\+?\s+(?=year)"
  matches=re.search(expression,str(x))
  if matches:
    result = matches.group(0)
  else:
      None
  
  #remove the '+' on some of the years of experience
  try:
    if result.split()[-1][-1] == '+':
      result = result.split()[-1].replace('+', '')
  except:
    result = result

  
  #convert to integer
  try:
    result = int(result)
  except:
    result = np.nan

  return result
  

data['year_of_exp'] = data['JobDesc'].apply(lambda x: years_exp(x))

data[['year_of_exp']].sample(10)

,year_of_exp
6395,NaN
1877,3.0
3561,2.0
664,2.0
2758,NaN
269,3.0
5118,4.0
7056,NaN
930,8.0
1858,5.0


### Contract Type Extraction
Extraction of the Contract Type

In [16]:
def contract_type(x):  
    
    """
    This function extracts the contract type from the scrapped job salary description.
    
    arg :
        x : the job description scrapped (str)
    
    return:
        results : contract type (str)
    """
    
    #find 'Type:' index location
    try:
        pos = str(x).split().index('Type:')
        
        #get a list of all text after "Type:"
        text = str(x)[pos+1 : ].split()
    except:
        text = []
    
    
    
    full_time_re = r'full-time\w*'
    contract_re = r'contract\w*'

    results = None

    for t in text:
        full_time_finder = re.findall(full_time_re, t.lower())
        contract_finder = re.findall(contract_re, t.lower())
        #hour_finder = re.findall(hour_re, t.lower())
        if len(full_time_finder) > 0:
            results = 'Full-time'
        elif len(contract_finder) > 0:
            results = 'Contract'

    return results


#extract the contract type for each row
data['contract_type'] = data['JobDesc'].apply(lambda x: contract_type(x))

data[['contract_type']].sample(10)

,contract_type
2042,None
2411,None
271,Full-time
6816,None
4504,None
4221,None
5141,None
1891,None
2954,None
8259,None
